<a href="https://colab.research.google.com/github/WakWakBird/MLS/blob/main/tf2-12-4-rnn_long_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [3]:
import tensorflow as tf
import numpy as np

# 입력 문장
sentence = (
    "if you want to build a ship, don't drum up people together to "
    "collect wood and don't assign them tasks and work, but rather "
    "teach them to long for the endless immensity of the sea."
)

# 고유 문자 집합 및 딕셔너리 생성
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

# 하이퍼파라미터
data_dim = len(char_set)             # 입력 차원 (문자 개수)
hidden_size = len(char_set)          # LSTM 유닛 수 (출력 클래스 수와 동일)
num_classes = len(char_set)          # 분류할 문자 수
sequence_length = 10
learning_rate = 0.1

# 시퀀스 데이터 생성
dataX = []
dataY = []
for i in range(len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1:i + sequence_length + 1]

    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]

    dataX.append(x)
    dataY.append(y)

# 텐서 형식으로 변환 (One-hot)
X_one_hot = tf.one_hot(dataX, num_classes)   # (N, seq_len, num_classes)
Y_one_hot = tf.one_hot(dataY, num_classes)   # (N, seq_len, num_classes)

# 모델 구성
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=num_classes, input_shape=(sequence_length, num_classes), return_sequences=True),
    tf.keras.layers.LSTM(units=num_classes, return_sequences=True),
    tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(units=num_classes, activation='softmax')
    )
])

# 컴파일
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    metrics=['accuracy']
)

model.summary()

# 학습
model.fit(X_one_hot, Y_one_hot, epochs=100, verbose=2)

# 예측
results = model.predict(X_one_hot)
for t, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if t == 0:
        print(''.join([char_set[i] for i in index]), end='')
    else:
        print(char_set[index[-1]], end='')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 25)         │         5,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 10, 25)         │         5,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 10, 25)         │           650 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,850 (42.38 KB)

 Trainable params: 10,850 (42.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
6/6 - 8s - 1s/step - accuracy: 0.1447 - loss: 3.0266
Epoch 2/100
6/6 - 0s - 20ms/step - accuracy: 0.1894 - loss: 2.8418
Epoch 3/100
6/6 - 0s - 24ms/step - accuracy: 0.2118 - loss: 2.6118
Epoch 4/100
6/6 - 0s - 14ms/step - accuracy: 0.3412 - loss: 2.2145
Epoch 5/100
6/6 - 0s - 24ms/step - accuracy: 0.4512 - loss: 1.8009
Epoch 6/100
6/6 - 0s - 23ms/step - accuracy: 0.5447 - loss: 1.4496
Epoch 7/100
6/6 - 0s - 24ms/step - accuracy: 0.6388 - loss: 1.1788
Epoch 8/100
6/6 - 0s - 23ms/step - accuracy: 0.7053 - loss: 0.9302
Epoch 9/100
6/6 - 0s - 17ms/step - accuracy: 0.7765 - loss: 0.7452
Epoch 10/100
6/6 - 0s - 15ms/step - accuracy: 0.7900 - loss: 0.6333
Epoch 11/100
6/6 - 0s - 23ms/step - accuracy: 0.8200 - loss: 0.5731
Epoch 12/100
6/6 - 0s - 14ms/step - accuracy: 0.8347 - loss: 0.5087
Epoch 13/100
6/6 - 0s - 14ms/step - accuracy: 0.8453 - loss: 0.4473
Epoch 14/100
6/6 - 0s - 15ms/step - accuracy: 0.8576 - loss: 0.3971
Epoch 15/100
6/6 - 0s - 23ms/step - accuracy: 0.8624 - loss